# Next_month_price_prediction_Alkalis

## TO-DOs
```
[v] Import monthly electrcity data
[v] Import monthly TTF_GAS data
[v] Import price evaluatioin data
[v] Create rows and encoding Alkalis_RM02_0001, Alkalis_RM02_0002
[] Create 12 features, external factor prices from one-month before to 12-month before, and encoding datetime data
[] To create a column indicating how many monthes was shifted, 'Monthes_shifted'
[] To create dummy variables for 'Monthes_shifted'
[] Combine features with target variables
[] Data scaling
[] check multicollinearity(to run one regression using each features, and find corr of all feature, filtering those with higher performance and least corr for our last model)
[] train_test_split()
[] Lasso regression
[] Cross validation
```

In [2]:
!pip install fredapi
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
from fredapi import Fred
from pandasql import sqldf
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def monthly_mean_to_daily(df_monthly: pd.core.frame.DataFrame ) -> pd.core.frame.DataFrame:
    """
    Convert Monthly data into Daily data and impute with monthly mean prices
    """
    df_monthly['Date'] = pd.to_datetime(df_monthly[['Year', 'Month']].assign(DAY=1))
    df = df_monthly.explode('Date') # The explode() method converts each element of the specified column(s) into a row.

    # Generate a complete range of daily dates for the year for imputation
    start_date = df['Date'].min() # represents the starting point of your data
    end_date = df['Date'].max() + pd.offsets.MonthEnd(1)  # finds the maximum (or latest) date and include the last month fully
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D') # generates a fixed-frequency DatetimeIndex

    # Merge the full date range with the monthly averages to fill in all days
    df_full_date_range = pd.DataFrame(full_date_range, columns=['Date'])
    df = pd.merge(df_full_date_range, df_monthly, on='Date', how='left')
    df_daily = df.ffill(axis=0) # to fill the missing value based on last valid observation following index sequence
    return df_daily

In [5]:
## Import monthly electrcity data
elec_df = pd.read_csv('ELECTRICITY.csv').iloc[:,1:]
elec_df['Time'] = pd.to_datetime(elec_df['Year'].astype(str) + elec_df['Month'].astype(str), format='%Y%m')
elec_df = elec_df[elec_df['Year'].between(2011,2023)].reset_index().drop('index',axis=1)

print(elec_df.info())
print(elec_df.groupby(['Year']).count())
print(elec_df.isna().sum().sort_values()) # checking missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Year         156 non-null    int64         
 1   Month        156 non-null    int64         
 2   Electricity  156 non-null    float64       
 3   Time         156 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 5.0 KB
None
      Month  Electricity  Time
Year                          
2011     12           12    12
2012     12           12    12
2013     12           12    12
2014     12           12    12
2015     12           12    12
2016     12           12    12
2017     12           12    12
2018     12           12    12
2019     12           12    12
2020     12           12    12
2021     12           12    12
2022     12           12    12
2023     12           12    12
Year           0
Month          0
Electricity    0
Time        

In [6]:
## Import monthly TTF_GAS data
apiKey = '29219060bc68b2802af8584e0f328b52'
fred = Fred(api_key=apiKey)

# Get Natural Gas prices in Europe per month
TTF_gas_df = pd.DataFrame(fred.get_series('PNGASEUUSDM'), 
                       columns=['PNGASEUUSDM']).reset_index() 
TTF_gas_df['index'] = pd.to_datetime(TTF_gas_df['index'], format='%Y-%m-%d')
TTF_gas_df['Year'] = TTF_gas_df['index'].dt.year
TTF_gas_df['Month'] = TTF_gas_df['index'].dt.month
TTF_gas_df = TTF_gas_df[TTF_gas_df['Year'].between(2011,2023)]
TTF_gas_df.rename(columns = {'index':'Time'}, inplace = True)

print(TTF_gas_df.info())
print(TTF_gas_df.groupby(['Year']).count())
print(TTF_gas_df.isna().sum().sort_values()) # Check missing values




<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 312 to 467
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Time         156 non-null    datetime64[ns]
 1   PNGASEUUSDM  156 non-null    float64       
 2   Year         156 non-null    int64         
 3   Month        156 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 6.1 KB
None
      Time  PNGASEUUSDM  Month
Year                          
2011    12           12     12
2012    12           12     12
2013    12           12     12
2014    12           12     12
2015    12           12     12
2016    12           12     12
2017    12           12     12
2018    12           12     12
2019    12           12     12
2020    12           12     12
2021    12           12     12
2022    12           12     12
2023    12           12     12
Time           0
PNGASEUUSDM    0
Year           0
Month     

In [7]:
## Import price evaluatioin data
price_evo_df = pd.read_csv('Dataset_Predicting_Price_Evolutions.csv').iloc[:,1:]
price_evo_df['POSTING DATE'] = pd.to_datetime(price_evo_df['POSTING DATE'], format='%Y-%m-%d')
price_evo_df['Year'] = price_evo_df['POSTING DATE'].dt.year
price_evo_df['Month'] = price_evo_df['POSTING DATE'].dt.month
# price_evo_df = price_evo_df.sort_values(['Year','Month'],ascending=True)
price_evo_df = price_evo_df[price_evo_df['Year'].between(2012,2023)].reset_index().drop(['index'], axis=1)

price_evo_df.rename(columns = {'POSTING DATE':'Time'}, inplace = True)

# Drop unnecessary columns
price_evo_df = price_evo_df.drop(['SITE', 'SUPPLIER NUMBER', 'PURCHASE NUMBER', 'WEIGHT (kg)'], axis=1)

print(price_evo_df.info())
print(price_evo_df.groupby(['Year']).count())
print(price_evo_df.isna().sum().sort_values()) # Check missing values



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20570 entries, 0 to 20569
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Time               20570 non-null  datetime64[ns]
 1   Group Description  20570 non-null  object        
 2   Key RM code        20570 non-null  object        
 3   PRICE (EUR/kg)     20570 non-null  float64       
 4   Year               20570 non-null  int64         
 5   Month              20570 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 964.3+ KB
None
      Time  Group Description  Key RM code  PRICE (EUR/kg)  Month
Year                                                             
2012   604                604          604             604    604
2013   634                634          634             634    634
2014   803                803          803             803    803
2015   860                860     

In [20]:
## Create rows and encoding Alkalis_RM02_0001, Alkalis_RM02_0002
Alkalis_df = price_evo_df[price_evo_df['Group Description']=="Alkalis"].sort_values(['Year','Month'],ascending=True)
Alkalis_df = Alkalis_df.reset_index().drop('index',axis=1)

# encoding Alkalis_RM02_0001, Alkalis_RM02_0002 with n-1 dummy variables
Alkalis_df_dummies = pd.get_dummies(Alkalis_df['Key RM code'], drop_first=True)

# combine dummy variables with Alkalis_df
Alkalis_df_dummies = pd.concat([Alkalis_df, Alkalis_df_dummies], axis=1)
Alkalis_df_dummies = Alkalis_df_dummies.drop('Key RM code', axis=1)
print(Alkalis_df_dummies.info())
print(Alkalis_df_dummies.groupby(['Year']).count())
print(Alkalis_df_dummies.sort_values('Time').head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007 entries, 0 to 6006
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Time               6007 non-null   datetime64[ns]
 1   Group Description  6007 non-null   object        
 2   PRICE (EUR/kg)     6007 non-null   float64       
 3   Year               6007 non-null   int64         
 4   Month              6007 non-null   int64         
 5   RM02/0002          6007 non-null   uint8         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), uint8(1)
memory usage: 240.6+ KB
None
      Time  Group Description  PRICE (EUR/kg)  Month  RM02/0002
Year                                                           
2012   148                148             148    148        148
2013   182                182             182    182        182
2014   219                219             219    219        219
2015   231                231       

In [23]:
## Create 12 features, external factor prices from one-month before to 12-month before, and encoding datetime data
# To merge TTF_gas and Electricity prices
df_merged = pd.merge(elec_df, TTF_gas_df,how='left', on = (['Year', 'Month', 'Time']))


# to create time shifts
# The pd.DateOffset is a pandas function that allows you to perform date arithmetic.
shifts = [1]
for i in shifts:
    df = df_merged
    mask = df['Time'] - pd.DateOffset(months=i)
    df_shifted = df[df['Time'].isin(mask)]

    # To join Alkalis_df_dummies on 'Shifted_time', changing type() from datetime to string to align the values
    df_shifted['Time'] = df_shifted['Time'].dt.strftime('%Y-%m')
    df_shifted.rename(columns = {'Time':'Shifted_time',
                                'Electricity':'His_Electricity',
                                'PNGASEUUSDM':'His_PNGASEUUSDM'}, inplace = True)
    df_shifted = df_shifted.drop(['Year', 'Month'], axis=1)
    Alkalis_df_dummies['Shifted_time'] = Alkalis_df_dummies['Time'] - pd.DateOffset(months=i)
    Alkalis_df_dummies['Shifted_time'] = Alkalis_df_dummies['Shifted_time'].dt.strftime('%Y-%m')
    
    print(Alkalis_df_dummies.sort_values('Shifted_time').head(20))
    print(df_shifted.sort_values('Shifted_time').head(20))
    
    test = pd.merge(Alkalis_df_dummies, df_shifted,how='left', on = (['Shifted_time']))
    print(test.isna().sum().sort_values())
    print(test.info())
    print(test.head(40))

# To-Dos, to create a loop to auto create time shifts from 1 to 12, with corresponding historical price, and auto merge into Alkalis_df_dummies
# To create a column indicating how many monthes was shifted, 'Monthes_shifted'
# To create dummy variables for 'Monthes_shifted'

         Time Group Description  PRICE (EUR/kg)  ...  Month  RM02/0002  Shifted_time
0  2012-01-31           Alkalis           0.215  ...      1          0       2011-12
1  2012-01-31           Alkalis           0.215  ...      1          0       2011-12
2  2012-01-31           Alkalis           0.217  ...      1          0       2011-12
3  2012-01-31           Alkalis           0.215  ...      1          0       2011-12
4  2012-01-31           Alkalis           0.205  ...      1          0       2011-12
5  2012-01-31           Alkalis           0.217  ...      1          0       2011-12
6  2012-01-31           Alkalis           0.215  ...      1          0       2011-12
7  2012-01-31           Alkalis           0.215  ...      1          0       2011-12
16 2012-02-24           Alkalis           0.410  ...      2          1       2012-01
15 2012-02-29           Alkalis           0.215  ...      2          0       2012-01
14 2012-02-29           Alkalis           0.217  ...      2      